<a href="https://colab.research.google.com/github/JavierSanLorenzoVIU/03MAIR---Algoritmos-de-Optimizacion---2024/blob/main/Trabajo_Practico_Javier_SanLorenzo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Algoritmos de optimización - Trabajo Práctico<br>
Nombre y Apellidos: Javier San Lorenzo Gómez  <br>
Url GIT: https://github.com/JavierSanLorenzoVIU/03MAIR---Algoritmos-de-Optimizacion---2024/blob/main/Trabajo_Practico_Javier_SanLorenzo.ipynb<br>

Problema:
>1. Sesiones de doblaje <br>
>2. Organizar los horarios de partidos de La Liga<br>
>3. Configuración de Tribunales                                        

---

###Descripción del problema:
- # Organizar los horarios de partidos de La Liga

---

In [11]:
import pandas as pd
import numpy as np
import random

# Modelo

### Representación del espacio de soluciones y enfoque del modelo:

El conjunto de soluciones abarca todas las asignaciones potenciales de horarios para los encuentros de fútbol, cada una representando una combinación única de horarios [ (nº de partidos)! ]. Para representar los datos y resultados se emplearán DataFrames de Pandas.


- ### **Enfoque**:


El enfoque del modelo se basa en desarrollar un **algoritmo voraz**.
La estrategia básica en estos algoritmos implica tomar decisiones en cada etapa para construir una solución óptima en ese momento, sin tener en cuenta las consecuencias futuras, descartando así otras posibles soluciones.

La agresividad de estos algoritmos los hace eficientes en muchos problemas donde tomar la mejor decisión en cada paso conduce a la mejor solución final. Sin embargo, esta eficacia no se garantiza en todos los casos.

Es importante destacar que la mejor solución en cada etapa no necesariamente conduce a la mejor solución final, lo que añade complejidad al proceso. Además, seleccionar una función adecuada de selección es crucial para el éxito del algoritmo voraz.

Además, para implementar este algoritmo se necesitará definir la función objetivo.

- ### **Función Objetivo**:
La **función objetivo** se encarga de calcular el valor que se busca maximizar o minimizar en el problema. En este caso: maximizar la audiencia de la jornada de partidos considerando los horarios y las coincidencias horarias.

$$ \text{AudienciaTotal} = \sum_{i=1}^{N} \text{Audiencia}_i \times \text{PonderacionHorario}_i \times (1- \text{CoefCoincidenciaPartidos}_i) $$

  * **N** → número de partidos.
  * **Audiencia** → audiencia del partido \(i\). Es la suma de las audiencias de cada equipo, donde su valor puede ser: 1, 2, 3, 4 o 5 (por cada equipo). (cuanto mayor el valor -> mayor la audiencia [máximo 10])
  * **PonderacionHorario** → coeficiente del horario en el que se juega el partido \(i\). (de 0 a 1)
  * **CoefCoincidenciaPartidos** → coeficiente de reducción de audiencia al existir varios partidos a la misma hora. (de 0 a 1)



### Restricciones:

  * Cada partido sólo se puede jugar una vez en la jornada.
  * Sólo hay partidos entre equipos de la misma división.





#Análisis
### ¿Que complejidad tiene el problema?. Orden de complejidad y contabilizar el espacio de soluciones.


Los datos se componen de:
1. Un data frame con 24 equipos y 3 divisiones, es decir, 8 equipos por división (con la puntuación de la audiencia).



In [14]:
clubs = ['Real Madrid','FC Barcelona','Atlético de Madrid','Sevilla FC','Valencia CF','Athletic Club de Bilbao','Real Sociedad','Villarreal CF',
         'Málaga CF','Real Oviedo','RC Deportivo','UD Las Palmas','Sporting de Gijón','Rayo Vallecano','CD Tenerife','AD Alcorcón',
         'CF Internacional de Madrid','CD Teruel','SD Logroñés','CP Cacereño','UB Conquense','UD Socuéllamos','CD Atlético Baleares','Real Murcia CF']

division = ['1','1','1','1','1','1','1','1',
            '2','2','2','2','2','2','2','2',
            '3','3','3','3','3','3','3','3']

# Afición/audiencia de cada club (de 1 a 5)
audiencia = ['5','5','5','4','5','4','3','3',
           '3','2','2','2','3','3','2','2',
           '1','2','2','1','1','1','1','3']

equipos = pd.DataFrame({'División': division, 'Audiencia (pts)': audiencia}, index=clubs)
display(equipos)

,División,Audiencia (pts)
Real Madrid,1,5
FC Barcelona,1,5
Atlético de Madrid,1,5
Sevilla FC,1,4
Valencia CF,1,5
Athletic Club de Bilbao,1,4
Real Sociedad,1,3
Villarreal CF,1,3
Málaga CF,2,3
Real Oviedo,2,2


2. Un data frame con los horarios que hay disponibles para los partidos (con su correspondiente coeficiente de ponderación).




In [15]:
ponderacion_horarios = pd.DataFrame({'Horario': ['V21:00h', 'S14:00h', 'S16:15h', 'S18:30h', 'S21h', 'D14:00h', 'D16:15h', 'D18:30h', 'D21h', 'L21:00h'], '%Audiencia': [0.75, 0.65, 0.7, 0.85, 1, 0.65, 0.7, 0.85, 1, 0.7]})
display(ponderacion_horarios)

,Horario,%Audiencia
0,V21:00h,0.75
1,S14:00h,0.65
2,S16:15h,0.70
3,S18:30h,0.85
4,S21h,1.00
5,D14:00h,0.65
6,D16:15h,0.70
7,D18:30h,0.85
8,D21h,1.00
9,L21:00h,0.70


3. Un data frame con los coeficientes de reducción de audiencia al coincidir varios partidos a la misma hora (con su correspondiente coeficiente de reducción de audiencia).



In [16]:
coincidencia_partidos = pd.DataFrame({'Coincidencia': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11], '%Audiencia': [0, 0.25, 0.4, 0.5, 0.55, 0.6, 0.65, 0.7, 0.75, 0.8, 0.85, 0.9]})
display(coincidencia_partidos)

,Coincidencia,%Audiencia
0,0,0.00
1,1,0.25
2,2,0.40
3,3,0.50
4,4,0.55
5,5,0.60
6,6,0.65
7,7,0.70
8,8,0.75
9,9,0.80


La distribución de los partidos se va a realizar mediante el método de algoritmos voraces, disminuyendo la complejidad a O(nlogn). Si este mismo problema se enfocase con fuerza bruta, al haber gran cantidad de comparaciones para llegar a la solución más optima, su compljidad aumentaría notablemente, alcanzando una complejidad factorial.

Por tanto, se tienen 24 equipos, lo que conformarán 12 partidos, y por ello habrá 12! soluciones que es igual a 479001600 posibles soluciones existentes.

#Diseño
### ¿Que técnica utilizo? ¿Por qué?

En cuanto al diseño del enfoque, se ha optado por emplear una técnica de algoritmos voraces para abordar el problema. En este sentido, se descompone el problema en diversas etapas, escogiendo en cada una la solución óptima que maximice la audiencia. Cada etapa corresponde a un partido específico, con el objetivo de maximizar su audiencia. Esta aproximación garantiza que cada partido se programe una sola vez, buscando optimizar la audiencia total a través de maximizar la de cada partido.

Inicialmente se generan los emparejamientos entre equipos de la misma división de manera aleatoria para formar los partidos.
[Las audiencias de los partidos tendrán un valor entre 2 y 10 puntos]



Después, se procede a ordenar el conjunto de datos de audiencia de menor a mayor. De esta manera, se minimiza la pérdida de audiencia.

En situaciones donde varios partidos alcanzan su máxima audiencia en el mismo horario, la audiencia de cada uno de estos partidos coincidentes se verá afectada.

In [17]:
# Creamos un DataFrame vacío para los enfrentamientos
partidos_jornada = pd.DataFrame(columns=['Equipo Local', 'Equipo Visitante', 'División', 'Audiencia'])

# Iteramos sobre cada división para crear los enfrentamientos
for division in equipos['División'].unique():
    equipos_division = equipos[equipos['División'] == division].index.tolist()
    np.random.shuffle(equipos_division)  # Barajamos aleatoriamente los equipos
    num_equipos = len(equipos_division)

    # Creamos enfrentamientos entre equipos de la misma división
    for i in range(0, num_equipos, 2):
        if i + 1 < num_equipos:
            equipo_local = equipos_division[i]
            equipo_visitante = equipos_division[i + 1]
            audiencia_total = int(equipos.loc[equipo_local, 'Audiencia (pts)']) + int(equipos.loc[equipo_visitante, 'Audiencia (pts)'])
            partidos_jornada = pd.concat([partidos_jornada, pd.DataFrame({'Equipo Local': [equipo_local],
                                                                       'Equipo Visitante': [equipo_visitante],
                                                                       'División': [division],
                                                                       'Audiencia': [audiencia_total]})], ignore_index=True)
display(partidos_jornada)

,Equipo Local,Equipo Visitante,División,Audiencia
0,Athletic Club de Bilbao,Real Sociedad,1,7
1,Real Madrid,Atlético de Madrid,1,10
2,Valencia CF,Sevilla FC,1,9
3,FC Barcelona,Villarreal CF,1,8
4,Real Oviedo,Málaga CF,2,5
5,Rayo Vallecano,AD Alcorcón,2,5
6,Sporting de Gijón,CD Tenerife,2,5
7,UD Las Palmas,RC Deportivo,2,4
8,SD Logroñés,CD Atlético Baleares,3,3
9,CD Teruel,UB Conquense,3,3


In [18]:
# Ordenar el DataFrame de menor a mayor según la audiencia de cada partido
partidos_jornada_ordenados = partidos_jornada.sort_values(by='Audiencia', ascending=True)

In [19]:
# Función para asignar los partidos a los horarios
def asignar_partidos(partidos_jornada_ordenados, pond_horarios, coef_coincidencia):
    horarios_asignados = pd.Series({'V21:00h': 0, 'S14:00h': 0, 'S16:15h': 0, 'S18:30h': 0, 'S21h': 0, 'D14:00h': 0, 'D16:15h': 0, 'D18:30h': 0, 'D21h': 0, 'L21:00h': 0})
    audiencias_finales = pd.DataFrame(columns=['Horario', 'Audiencia'])
    audiencia_total = 0

    for index, partido in partidos_jornada_ordenados.iterrows():
        mejor_audiencia = 0
        mejor_horario = None
        num_coinci = 0

        for horario in pond_horarios['Horario']:
            num_coinci = horarios_asignados[horario]
            audiencia_actual = pond_horarios.loc[pond_horarios['Horario'] == horario, '%Audiencia'].iloc[0] * partido['Audiencia'] * (1 - coef_coincidencia.loc[num_coinci, '%Audiencia'])
            if audiencia_actual > mejor_audiencia:
                mejor_audiencia = audiencia_actual
                mejor_horario = horario

        horarios_asignados[mejor_horario] += 1
        num_coinci = horarios_asignados[mejor_horario]

        #Si ha coincidencias se resta el coeficiente de la audiencia correspondiente al partido asignado inicialmente en ese horario
        if num_coinci > 1:
            repe = audiencias_finales[audiencias_finales['Horario'] == mejor_horario]
            audiencias_finales.loc[repe.index, 'Audiencia'] *= (1 - coef_coincidencia.loc[num_coinci, '%Audiencia'])
            audiencia_total = audiencia_total - repe.loc[repe.index, 'Audiencia'].iloc[0] + audiencias_finales.loc[repe.index, 'Audiencia'].iloc[0]

        audiencia_total += mejor_audiencia
        audiencias_finales.loc[index] = [mejor_horario, mejor_audiencia]

    return audiencia_total, audiencias_finales

# Asignar partidos a horarios
audiencia_total, audiencias_finales = asignar_partidos(partidos_jornada_ordenados, ponderacion_horarios, coincidencia_partidos)

print("\nAudiencia total:", audiencia_total,"\n")
audiencias_finales.insert(0, 'Partido', partidos_jornada_ordenados['Equipo Local'] + " - " + partidos_jornada_ordenados['Equipo Visitante'])
display(audiencias_finales)




Audiencia total: 45.8 



,Partido,Horario,Audiencia
11,CF Internacional de Madrid - CP Cacereño,S21h,1.20
8,SD Logroñés - CD Atlético Baleares,D21h,1.80
9,CD Teruel - UB Conquense,S18:30h,2.55
7,UD Las Palmas - RC Deportivo,D18:30h,3.40
10,Real Murcia CF - UD Socuéllamos,V21:00h,3.00
4,Real Oviedo - Málaga CF,S21h,3.75
5,Rayo Vallecano - AD Alcorcón,D21h,3.75
6,Sporting de Gijón - CD Tenerife,S16:15h,3.50
0,Athletic Club de Bilbao - Real Sociedad,D16:15h,4.90
3,FC Barcelona - Villarreal CF,L21:00h,5.60
